In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

In [2]:
# Key.json contains information for service account to connect to BigQuery API
credentials = service_account.Credentials.from_service_account_file('key.json')
project_id = 'zillow-houses-376821'
client = bigquery.Client(credentials = credentials, project = project_id)

In [21]:
# https://medium.com/pipeline-a-data-engineering-resource/automate-your-bigquery-schema-definitions-with-5-lines-of-python-7a1996749718
# Automate schema creation

def create_schema(field_list:list, type_list:list):
    schema_list = []
    for fields, types in zip(field_list, type_list):
        schema = bigquery.SchemaField(fields, types)
        schema_list.append(schema)
    return schema_list

# Function to load dataframe into BigQuery

def bq_load(df, dataset_id: str, table_id: str, schema, client):
    bq_client = client
    dataset_ref = bq_client.dataset(dataset_id)
    dataset_table_id = dataset_ref.table(table_id)
    
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition='WRITE_TRUNCATE'
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect=False
    # Calling create_schema()
    job_config.schema = schema
    job_config.ignore_unknown_values=False
    job = client.load_table_from_dataframe(df,
    table_id,
    location='US',
    job_config=job_config)
    
    return job.result()

In [ ]:
# Manual schema creation

# schema = [
#     bigquery.SchemaField("zpid", "INTEGER"),
#     bigquery.SchemaField("streetAddress", "STRING"),
#     bigquery.SchemaField("zipcode", "STRING"),
#     bigquery.SchemaField("city", "STRING"),
#     bigquery.SchemaField("state","STRING"),
#     bigquery.SchemaField("latitude", "NUMERIC"),
#     bigquery.SchemaField("longitude", "NUMERIC"),
#     bigquery.SchemaField("price", "INTEGER"),
#     bigquery.SchemaField("bathrooms", "FLOAT64"),
#     bigquery.SchemaField("bedrooms", "FLOAT64"),
#     bigquery.SchemaField("livingArea", "FLOAT64"),
#     bigquery.SchemaField("homeType", "STRING"),
#     bigquery.SchemaField("taxAssessedValue", "FLOAT64"),
#     bigquery.SchemaField("brokerName", "STRING")
# ]

In [43]:
# df_to_bq = bq_load(df, "zillow_houses", "zillow-houses-376821.zillow_houses.vancouverhouses", create_schema(field_list,type_list), client)

In [10]:
df = pd.read_csv('data/reviseddataset.csv')

In [11]:
df = df.rename(columns = {'newAddress':'addressWithoutUnit'})

In [12]:
df.columns

Index(['zpid', 'streetAddress', 'zipcode', 'city', 'state', 'latitude',
       'longitude', 'price', 'bathrooms', 'bedrooms', 'livingArea', 'homeType',
       'taxAssessedValue', 'lotAreaValue', 'brokerName', 'neighbourhoodName',
       'addressWithoutUnit'],
      dtype='object')

In [50]:
# df = df.drop(219)

In [18]:
location = df[['zpid', 'streetAddress', 'zipcode', 'city', 'state', 'latitude', 'longitude','addressWithoutUnit']]
homeinfo = df[['streetAddress', 'price', 'bathrooms', 'bedrooms', 'livingArea', 'homeType', 'taxAssessedValue', 'lotAreaValue', 'neighbourhoodName']]
brokerageinfo = df[['zpid','brokerName']]

In [19]:
# Gathering the types of each column and appending BigQuery datatype to list

def type_list_generator(df):
    type_list = []

    for column in df.columns:
        if df[column].dtype == 'object':
            type_list.append("STRING")
        elif df[column].dtype == 'float64':
            type_list.append("FLOAT64")
        else:
            type_list.append("INTEGER")
    return type_list

# Gathering every column name
def field_list_generator(df):
    field_list = list(df.columns)
    return field_list

In [53]:
# load relations to bigQuery

location_to_bq = bq_load(location, "zillow_houses", "zillow-houses-376821.zillow_houses.location", 
                         create_schema(field_list_generator(location),type_list_generator(location)), client)

homeinfo_to_bq = bq_load(homeinfo, "zillow_houses", "zillow-houses-376821.zillow_houses.homeinfo", 
                         create_schema(field_list_generator(homeinfo),type_list_generator(homeinfo)), client)

brokerageinfo_to_bq = bq_load(brokerageinfo, "zillow_houses", "zillow-houses-376821.zillow_houses.brokerageinfo", 
                              create_schema(field_list_generator(brokerageinfo),type_list_generator(brokerageinfo)), client)

In [3]:
# Testing query

initial_query = """ 

SELECT * FROM `zillow-houses-376821.zillow_houses.location` LIMIT 10

"""

query_results = client.query(initial_query).to_dataframe()

print(query_results)

         zpid            streetAddress zipcode       city state   latitude  \
0   314396616           2973 McGill St  V5K1H8  Vancouver    BC  49.289010   
1   314397260           2565 Dundas St  V5K1P7  Vancouver    BC  49.285038   
2   314397306           2697 Dundas St  V5K1R1  Vancouver    BC  49.285027   
3   314398013         3466 Franklin St  V5K1Y3  Vancouver    BC  49.281788   
4   314398017         3467 Franklin St  V5K1Y4  Vancouver    BC  49.282303   
5   314398025         3495 Franklin St  V5K1Y4  Vancouver    BC  49.282300   
6  2059767952  2741 Hastings St E #303  V5K1Z8  Vancouver    BC  49.281414   
7  2059768478        3537 Georgia St E  V5K2J4  Vancouver    BC  49.278503   
8  2060865873     3523 Georgia St E #2  V5K2L9  Vancouver    BC  49.278477   
9   314399067           2447 Adanac St  V5K2M3  Vancouver    BC  49.277690   

    longitude   addressWithoutUnit  
0 -123.042404       2973 McGill St  
1 -123.052950       2565 Dundas St  
2 -123.049650       2697 Dunda

In [54]:
# Top 5 lowest $/sq ft per neighbourhood

top5lowestpricepersqft = """ 

WITH dollar_sqft_calculations AS (
    SELECT streetAddress, neighbourhoodName, 
    bathrooms, bedrooms,
    price, livingArea, homeType, round(price/livingArea,2) as dollarPerSqFt
    FROM `zillow-houses-376821.zillow_houses.homeinfo`
),
ranking_dollarpersq AS (
    SELECT *, 
    rank() over (PARTITION BY neighbourhoodName ORDER BY dollarPerSqFt) as ranking
    FROM dollar_sqft_calculations
)

SELECT * FROM ranking_dollarpersq WHERE ranking <= 5 

"""

query_results = client.query(top5lowestpricepersqft).to_dataframe()



In [55]:
print(query_results)

                   streetAddress neighbourhoodName  bathrooms  bedrooms  \
0            3445 E 49th Ave #98            Sunset        2.0       4.0   
1                1322 E 62nd Ave            Sunset        6.0       6.0   
2                1045 E 63rd Ave            Sunset        4.0       6.0   
3                 268 E 65th Ave            Sunset        3.0       5.0   
4                 320 E 57th Ave            Sunset        3.0       5.0   
..                           ...               ...        ...       ...   
104               1055 Wolfe Ave       Shaughnessy        5.0       8.0   
105           1056 Richelieu Ave       Shaughnessy        3.0       5.0   
106  1011 King Edward Ave W #201       Shaughnessy        2.0       2.0   
107                4549 Osler St       Shaughnessy        6.0       6.0   
108                5290 Angus Dr       Shaughnessy        4.0       6.0   

         price  livingArea       homeType  dollarPerSqFt  ranking  
0     833000.0      1393.0     

In [56]:
query_results.to_csv('queries/Top5LowestDollarPerSqft.csv', index = False)

In [57]:
# Top 3 highest price per neighbourhood

top3highestprice = """ 

WITH top_3_rank AS (
    SELECT streetAddress, neighbourhoodName, 
    bathrooms, bedrooms,
    price, livingArea, homeType, 
    round(price/livingArea,2) as dollarPerSqFt,
    rank() over (PARTITION BY neighbourhoodName ORDER BY price DESC) as ranking
    FROM `zillow-houses-376821.zillow_houses.homeinfo`
)

SELECT * FROM top_3_rank WHERE ranking <= 3 

"""

query2_results = client.query(top3highestprice).to_dataframe()

In [58]:
print(query2_results)

            streetAddress    neighbourhoodName  bathrooms  bedrooms  \
0        4838 Belmont Ave      West Point Grey        8.0       5.0   
1        4788 Belmont Ave      West Point Grey       11.0       7.0   
2        1784 Drummond Dr      West Point Grey        2.0       4.0   
3   2112 Prince Edward St       Mount Pleasant        4.0       3.0   
4           20 W 14th Ave       Mount Pleasant        2.0       4.0   
..                    ...                  ...        ...       ...   
62           3601 Rae Ave  Renfrew-Collingwood        3.0       3.0   
63        2647 E 21st Ave  Renfrew-Collingwood        2.0       4.0   
64        5038 Arbutus St        Arbutus-Ridge        9.0       6.0   
65        2475 W 34th Ave        Arbutus-Ridge        7.0       6.0   
66          4723 Puget Dr        Arbutus-Ridge        8.0       7.0   

         price  livingArea       homeType  dollarPerSqFt  ranking  
0   59800000.0     12410.0  SINGLE_FAMILY        4818.69        1  
1   3388800

In [59]:
query2_results.to_csv('queries/Top3HighestPrice.csv', index = False)

In [60]:
# Top 3 lowest price per neighbourhood

top3lowestprice = """ 

WITH top_3_rank AS (
    SELECT streetAddress, neighbourhoodName, 
    bathrooms, bedrooms,
    price, livingArea, homeType, 
    round(price/livingArea,2) as dollarPerSqFt,
    rank() over (PARTITION BY neighbourhoodName ORDER BY price) as ranking
    FROM `zillow-houses-376821.zillow_houses.homeinfo`
)

SELECT * FROM top_3_rank WHERE ranking <= 3 

"""

query3_results = client.query(top3lowestprice).to_dataframe()

In [61]:
print(query3_results)

               streetAddress   neighbourhoodName  bathrooms  bedrooms  \
0       1055 Broadway E #204            Fairview        1.0       1.0   
1       1551 W 11th Ave #208            Fairview        1.0       1.0   
2       1216 W 11th Ave #206            Fairview        1.0       1.0   
3          2224 Eton St #103  Grandview-Woodland        1.0       0.0   
4        1533 E 8th Ave #101  Grandview-Woodland        1.0       1.0   
..                       ...                 ...        ...       ...   
62  489 Interurban Way #1505             Marpole        1.0       1.0   
63   489 Interurban Way #710             Marpole        1.0       1.0   
64      5926 Tisdall St #506            Oakridge        1.0       2.0   
65      2146 W 43rd Ave #201            Oakridge        1.0       2.0   
66          5770 Oak St #212            Oakridge        1.0       1.0   

       price  livingArea homeType  dollarPerSqFt  ranking  
0   550000.0       518.0    CONDO        1061.78        1  
1  

In [62]:
query3_results.to_csv('queries/Top3LowestPrice.csv', index = False)

In [63]:
# Brokerage and total number of units they list, avg price, top neighbourhood listed

brokeragedetails = """ 

WITH brokerage_count_avg AS (
    SELECT bi.brokerName, count(*) as number_of_listings, 
    avg(hi.price) as average_listing_price
    FROM `zillow-houses-376821.zillow_houses.brokerageinfo` bi JOIN
    `zillow-houses-376821.zillow_houses.location` l 
    ON bi.zpid = l.zpid
    JOIN `zillow-houses-376821.zillow_houses.homeinfo` hi 
    ON l.streetAddress = hi.streetAddress
    GROUP BY bi.brokerName
),
brokerage_neighbourhood AS (
    SELECT bi.brokerName, hi.neighbourhoodName, 
    count(hi.neighbourhoodName) as number_of_houses
    FROM `zillow-houses-376821.zillow_houses.brokerageinfo` bi JOIN
    `zillow-houses-376821.zillow_houses.location` l 
    ON bi.zpid = l.zpid
    JOIN `zillow-houses-376821.zillow_houses.homeinfo` hi 
    ON l.streetAddress = hi.streetAddress
    group by bi.brokerName, hi.neighbourhoodName
),
brokerage_n_rank AS (
    SELECT 
        *,
        rank() over (partition by brokerName 
        order by number_of_houses DESC) AS ranking
    FROM brokerage_neighbourhood 
)

SELECT 
    bca.brokerName, bn.neighbourhoodName as top_listed_neighbourhood,
    bca.number_of_listings AS total_listings, bca.average_listing_price,
    bn.number_of_houses as listing_in_neighbourhood
    FROM brokerage_count_avg bca JOIN brokerage_n_rank bn
    ON bca.brokerName = bn.brokerName
    WHERE bn.ranking = 1
    ORDER BY number_of_listings DESC
"""

query4_results = client.query(brokeragedetails).to_dataframe()

In [64]:
print(query4_results)

                                   brokerName top_listed_neighbourhood  \
0               RE/MAX Crest Realty Brokerage                 Downtown   
1    Sutton Group-West Coast Realty Brokerage                 Downtown   
2                Oakwyn Realty Ltd. Brokerage                 Downtown   
3                  Macdonald Realty Brokerage                 Downtown   
4     Stilhavn Real Estate Services Brokerage                 Downtown   
..                                        ...                      ...   
183           RE/MAX Elevate Realty Brokerage                 Downtown   
184       Argus Estates (1983) Ltd. Brokerage      Renfrew-Collingwood   
185               Umber Realty Inc. Brokerage       Grandview-Woodland   
186             SRS Westside Realty Brokerage                   Sunset   
187             eXp Realty (Branch) Brokerage      Renfrew-Collingwood   

     total_listings  average_listing_price  listing_in_neighbourhood  
0                70           2.440918e+

In [65]:
query4_results.to_csv('queries/BrokerageDetails.csv', index = False)

In [66]:
# Top 3 bathrooms/bedrooms per sq ft per neighbourhood

Top3BdBaPerSqft = """ 

WITH calculations AS (
    SELECT
        streetAddress, price, bathrooms, bedrooms, livingArea, homeType,
        bathrooms/livingArea as ba_per_sqft, bedrooms/livingArea as br_per_sqft,
        neighbourhoodName
    FROM `zillow-houses-376821.zillow_houses.homeinfo`
),
ranking as (
    SELECT 
        *, rank() OVER (PARTITION BY neighbourhoodName 
            ORDER BY ba_per_sqft DESC) as ranking_ba,
        rank() OVER (PARTITION BY neighbourhoodName 
            ORDER BY br_per_sqft DESC) as ranking_br
    FROM calculations
)

SELECT 
    * FROM ranking WHERE ranking_ba <= 3 or ranking_br <= 3


"""

query5_results = client.query(Top3BdBaPerSqft).to_dataframe()

In [67]:
print(query5_results)

            streetAddress      price  bathrooms  bedrooms  livingArea  \
0         2843 E 20th Ave  1759000.0        3.0       8.0      2065.0   
1      5058 Joyce St #401   928000.0        2.0       3.0       961.0   
2            3651 Rae Ave  1250000.0        3.0       4.0      1300.0   
3         2469 E 40th Ave  1399000.0        4.0       4.0      1437.0   
4    4770 Duchess St #SL1  1049000.0        3.0       3.0      1086.0   
..                    ...        ...        ...       ...         ...   
105  2083 W 33rd Ave #402  1288000.0        2.0       3.0      1200.0   
106       2849 W 18th Ave  2299000.0        4.0       5.0      2137.0   
107       2815 W 19th Ave  2278000.0        4.0       4.0      1780.0   
108    5626 Larch St #205   788000.0        2.0       2.0       962.0   
109         5408 Larch St  1399000.0        3.0       2.0      1156.0   

          homeType  ba_per_sqft  br_per_sqft    neighbourhoodName  ranking_ba  \
0    SINGLE_FAMILY     0.001453     0.0038

In [68]:
query5_results.to_csv('queries/Top3BdBaPerSqft.csv', index = False)

In [69]:
# Average price, number of listings per neighbourhood

AvgTotalNeighbourhood = """ 


SELECT
    neighbourhoodName, count(*) as total_listings, avg(price) as avg_price
FROM `zillow-houses-376821.zillow_houses.homeinfo`
GROUP BY neighbourhoodName


"""

query6_results = client.query(AvgTotalNeighbourhood).to_dataframe()

In [70]:
print(query6_results)

           neighbourhoodName  total_listings     avg_price
0                     Sunset              26  1.748780e+06
1                    Marpole              21  3.211694e+06
2                   Downtown             172  2.407403e+06
3                   Fairview              50  1.166800e+06
4                   Oakridge              20  3.853939e+06
5                   West End              62  2.174114e+06
6                  Killarney              27  9.708259e+05
7                  Kitsilano              54  3.730285e+06
8                 Kerrisdale              22  7.449991e+06
9                 Riley Park              18  2.099633e+06
10                Strathcona               4  1.133000e+06
11               Shaughnessy              37  9.838837e+06
12              South Cambie               8  2.789250e+06
13             Arbutus-Ridge              28  3.805313e+06
14            Mount Pleasant              33  1.215879e+06
15           West Point Grey              33  8.711324e+

In [71]:
query6_results.to_csv('queries/AvgTotalNeighbourhood.csv', index = False)

In [74]:
# Lowest price by combinations of bedroom and bathrooms

LowestPriceBrBa = """ 

WITH ranking_low_price AS (
SELECT
    streetAddress, bathrooms, bedrooms, price, 
    livingArea, homeType, neighbourhoodName,
    rank() OVER (PARTITION BY CAST(bathrooms as STRING), 
        CAST(bedrooms as STRING)
                        ORDER BY price) as ranking
FROM `zillow-houses-376821.zillow_houses.homeinfo`
)

SELECT 
    * 
    FROM ranking_low_price
    WHERE ranking = 1


"""

query7_results = client.query(LowestPriceBrBa).to_dataframe()

In [75]:
print(query7_results)

             streetAddress  bathrooms  bedrooms       price  livingArea  \
0          1188 W 55th Ave       12.0       8.0  28880000.0     15371.0   
1    289 Alexander St #808        2.0       1.0    798000.0       790.0   
2          2026 E 32nd Ave        7.0       8.0   2888000.0      2674.0   
3         1965 Ferndale St        0.0       3.0   1049000.0      1470.0   
4        6869 Beechwood St       10.0       8.0   9998000.0      7889.0   
..                     ...        ...       ...         ...         ...   
58  1415 W Georgia St #113        0.0       0.0    100000.0       200.0   
59         1067 E 14th Ave        2.0       5.0   1588000.0      2160.0   
60         3255 W 26th Ave        7.0       5.0   5980000.0      4442.0   
61          727 E 39th Ave        3.0       9.0   1700000.0      2880.0   
62        3206 Carolina St        5.0       6.0   2198000.0      2680.0   

         homeType         neighbourhoodName  ranking  
0   SINGLE_FAMILY                  Oakridge 

In [76]:
query7_results.to_csv('queries/LowestPriceBrBa.csv', index = False)